In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf

In [3]:
# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

In [4]:
# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

In [5]:
# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

In [6]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

In [7]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [8]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [9]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [10]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [11]:
# Model
logits = conv_net(x, weights, biases, keep_prob)

In [12]:
# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

In [13]:
# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [14]:
# Initializing the variables
init = tf. global_variables_initializer()

In [15]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 82219.9375 Validation Accuracy: 0.062500
Epoch  1, Batch   2 -Loss: 62935.6992 Validation Accuracy: 0.070312
Epoch  1, Batch   3 -Loss: 59076.2812 Validation Accuracy: 0.082031
Epoch  1, Batch   4 -Loss: 52329.8672 Validation Accuracy: 0.082031
Epoch  1, Batch   5 -Loss: 44530.6406 Validation Accuracy: 0.097656
Epoch  1, Batch   6 -Loss: 44715.8789 Validation Accuracy: 0.093750
Epoch  1, Batch   7 -Loss: 40386.7734 Validation Accuracy: 0.097656
Epoch  1, Batch   8 -Loss: 40158.2656 Validation Accuracy: 0.109375
Epoch  1, Batch   9 -Loss: 36460.1328 Validation Accuracy: 0.117188
Epoch  1, Batch  10 -Loss: 34233.4375 Validation Accuracy: 0.117188
Epoch  1, Batch  11 -Loss: 32288.9785 Validation Accuracy: 0.121094
Epoch  1, Batch  12 -Loss: 28586.3086 Validation Accuracy: 0.105469
Epoch  1, Batch  13 -Loss: 26655.2578 Validation Accuracy: 0.109375
Epoch  1, Batch  14 -Loss: 24604.9766 Validation Accuracy: 0.109375
Epoch  1, Batch  15 -Loss: 22866.4727 Validation

Epoch  1, Batch 122 -Loss:  3858.9297 Validation Accuracy: 0.566406
Epoch  1, Batch 123 -Loss:  4829.6475 Validation Accuracy: 0.570312
Epoch  1, Batch 124 -Loss:  4378.1450 Validation Accuracy: 0.578125
Epoch  1, Batch 125 -Loss:  4216.3730 Validation Accuracy: 0.574219
Epoch  1, Batch 126 -Loss:  3888.9497 Validation Accuracy: 0.582031
Epoch  1, Batch 127 -Loss:  4267.6074 Validation Accuracy: 0.574219
Epoch  1, Batch 128 -Loss:  3987.3833 Validation Accuracy: 0.589844
Epoch  1, Batch 129 -Loss:  4108.6875 Validation Accuracy: 0.578125
Epoch  1, Batch 130 -Loss:  4304.5840 Validation Accuracy: 0.582031
Epoch  1, Batch 131 -Loss:  4334.6074 Validation Accuracy: 0.582031
Epoch  1, Batch 132 -Loss:  3178.0552 Validation Accuracy: 0.585938
Epoch  1, Batch 133 -Loss:  3957.2566 Validation Accuracy: 0.585938
Epoch  1, Batch 134 -Loss:  3429.2800 Validation Accuracy: 0.593750
Epoch  1, Batch 135 -Loss:  3952.7764 Validation Accuracy: 0.597656
Epoch  1, Batch 136 -Loss:  4657.6445 Validation

Epoch  1, Batch 243 -Loss:  2791.8901 Validation Accuracy: 0.679688
Epoch  1, Batch 244 -Loss:  2828.6550 Validation Accuracy: 0.675781
Epoch  1, Batch 245 -Loss:  2686.4180 Validation Accuracy: 0.671875
Epoch  1, Batch 246 -Loss:  2916.7388 Validation Accuracy: 0.683594
Epoch  1, Batch 247 -Loss:  2869.8733 Validation Accuracy: 0.687500
Epoch  1, Batch 248 -Loss:  1949.9780 Validation Accuracy: 0.675781
Epoch  1, Batch 249 -Loss:  2873.9536 Validation Accuracy: 0.675781
Epoch  1, Batch 250 -Loss:  3219.4299 Validation Accuracy: 0.679688
Epoch  1, Batch 251 -Loss:  2349.0520 Validation Accuracy: 0.679688
Epoch  1, Batch 252 -Loss:  2356.8857 Validation Accuracy: 0.687500
Epoch  1, Batch 253 -Loss:  3486.8813 Validation Accuracy: 0.687500
Epoch  1, Batch 254 -Loss:  2357.5515 Validation Accuracy: 0.687500
Epoch  1, Batch 255 -Loss:  3094.8506 Validation Accuracy: 0.683594
Epoch  1, Batch 256 -Loss:  2546.3799 Validation Accuracy: 0.679688
Epoch  1, Batch 257 -Loss:  2195.7920 Validation

Epoch  1, Batch 364 -Loss:  2002.9448 Validation Accuracy: 0.718750
Epoch  1, Batch 365 -Loss:  1900.6248 Validation Accuracy: 0.722656
Epoch  1, Batch 366 -Loss:  1513.9910 Validation Accuracy: 0.722656
Epoch  1, Batch 367 -Loss:  2248.9492 Validation Accuracy: 0.714844
Epoch  1, Batch 368 -Loss:  1842.9805 Validation Accuracy: 0.718750
Epoch  1, Batch 369 -Loss:  1636.9741 Validation Accuracy: 0.718750
Epoch  1, Batch 370 -Loss:  1642.0814 Validation Accuracy: 0.726562
Epoch  1, Batch 371 -Loss:  2069.0461 Validation Accuracy: 0.722656
Epoch  1, Batch 372 -Loss:  1713.5945 Validation Accuracy: 0.718750
Epoch  1, Batch 373 -Loss:  1947.9603 Validation Accuracy: 0.722656
Epoch  1, Batch 374 -Loss:  2215.3071 Validation Accuracy: 0.738281
Epoch  1, Batch 375 -Loss:  1908.1050 Validation Accuracy: 0.730469
Epoch  1, Batch 376 -Loss:  1926.4082 Validation Accuracy: 0.730469
Epoch  1, Batch 377 -Loss:  1842.2935 Validation Accuracy: 0.726562
Epoch  1, Batch 378 -Loss:  1473.4351 Validation

Epoch  2, Batch  56 -Loss:  1808.0649 Validation Accuracy: 0.761719
Epoch  2, Batch  57 -Loss:  1165.0308 Validation Accuracy: 0.765625
Epoch  2, Batch  58 -Loss:  1519.4255 Validation Accuracy: 0.773438
Epoch  2, Batch  59 -Loss:  1839.2404 Validation Accuracy: 0.753906
Epoch  2, Batch  60 -Loss:  1482.7429 Validation Accuracy: 0.761719
Epoch  2, Batch  61 -Loss:  1754.4916 Validation Accuracy: 0.761719
Epoch  2, Batch  62 -Loss:  1415.8010 Validation Accuracy: 0.757812
Epoch  2, Batch  63 -Loss:  1907.9482 Validation Accuracy: 0.757812
Epoch  2, Batch  64 -Loss:  1363.5281 Validation Accuracy: 0.769531
Epoch  2, Batch  65 -Loss:  1199.9895 Validation Accuracy: 0.765625
Epoch  2, Batch  66 -Loss:  1456.2385 Validation Accuracy: 0.769531
Epoch  2, Batch  67 -Loss:  1136.2192 Validation Accuracy: 0.765625
Epoch  2, Batch  68 -Loss:  1716.7150 Validation Accuracy: 0.753906
Epoch  2, Batch  69 -Loss:  1425.7104 Validation Accuracy: 0.761719
Epoch  2, Batch  70 -Loss:  1302.5022 Validation

...

Epoch  9, Batch 320 -Loss:   277.5773 Validation Accuracy: 0.824219
Epoch  9, Batch 321 -Loss:   316.6380 Validation Accuracy: 0.820312
Epoch  9, Batch 322 -Loss:   282.6636 Validation Accuracy: 0.816406
Epoch  9, Batch 323 -Loss:   226.1301 Validation Accuracy: 0.820312
Epoch  9, Batch 324 -Loss:   246.0556 Validation Accuracy: 0.820312
Epoch  9, Batch 325 -Loss:   159.6910 Validation Accuracy: 0.816406
Epoch  9, Batch 326 -Loss:   235.3944 Validation Accuracy: 0.816406
Epoch  9, Batch 327 -Loss:   363.6513 Validation Accuracy: 0.820312
Epoch  9, Batch 328 -Loss:   247.6637 Validation Accuracy: 0.816406
Epoch  9, Batch 329 -Loss:   302.5243 Validation Accuracy: 0.812500
Epoch  9, Batch 330 -Loss:   319.6938 Validation Accuracy: 0.824219
Epoch  9, Batch 331 -Loss:   252.7477 Validation Accuracy: 0.812500
Epoch  9, Batch 332 -Loss:   288.0227 Validation Accuracy: 0.812500
Epoch  9, Batch 333 -Loss:   460.6084 Validation Accuracy: 0.812500
Epoch  9, Batch 334 -Loss:   206.6637 Validation

Epoch 10, Batch  12 -Loss:   304.5392 Validation Accuracy: 0.816406
Epoch 10, Batch  13 -Loss:   246.0438 Validation Accuracy: 0.816406
Epoch 10, Batch  14 -Loss:   189.1102 Validation Accuracy: 0.812500
Epoch 10, Batch  15 -Loss:   251.5727 Validation Accuracy: 0.812500
Epoch 10, Batch  16 -Loss:   347.4927 Validation Accuracy: 0.808594
Epoch 10, Batch  17 -Loss:   194.2214 Validation Accuracy: 0.812500
Epoch 10, Batch  18 -Loss:   295.3633 Validation Accuracy: 0.804688
Epoch 10, Batch  19 -Loss:   343.7674 Validation Accuracy: 0.812500
Epoch 10, Batch  20 -Loss:   386.0081 Validation Accuracy: 0.812500
Epoch 10, Batch  21 -Loss:   282.0673 Validation Accuracy: 0.820312
Epoch 10, Batch  22 -Loss:   255.6122 Validation Accuracy: 0.820312
Epoch 10, Batch  23 -Loss:   229.9327 Validation Accuracy: 0.820312
Epoch 10, Batch  24 -Loss:   278.1701 Validation Accuracy: 0.820312
Epoch 10, Batch  25 -Loss:   250.8854 Validation Accuracy: 0.812500
Epoch 10, Batch  26 -Loss:   303.5276 Validation

Epoch 10, Batch 133 -Loss:   230.0771 Validation Accuracy: 0.816406
Epoch 10, Batch 134 -Loss:   333.8472 Validation Accuracy: 0.816406
Epoch 10, Batch 135 -Loss:   196.7378 Validation Accuracy: 0.816406
Epoch 10, Batch 136 -Loss:   308.5075 Validation Accuracy: 0.812500
Epoch 10, Batch 137 -Loss:   223.1943 Validation Accuracy: 0.820312
Epoch 10, Batch 138 -Loss:   369.1871 Validation Accuracy: 0.812500
Epoch 10, Batch 139 -Loss:   291.8589 Validation Accuracy: 0.812500
Epoch 10, Batch 140 -Loss:   212.4988 Validation Accuracy: 0.808594
Epoch 10, Batch 141 -Loss:   264.0405 Validation Accuracy: 0.808594
Epoch 10, Batch 142 -Loss:   274.1731 Validation Accuracy: 0.804688
Epoch 10, Batch 143 -Loss:   363.7721 Validation Accuracy: 0.804688
Epoch 10, Batch 144 -Loss:   227.2483 Validation Accuracy: 0.804688
Epoch 10, Batch 145 -Loss:   280.9595 Validation Accuracy: 0.808594
Epoch 10, Batch 146 -Loss:   278.6765 Validation Accuracy: 0.812500
Epoch 10, Batch 147 -Loss:   468.1297 Validation

Epoch 10, Batch 254 -Loss:   244.0666 Validation Accuracy: 0.804688
Epoch 10, Batch 255 -Loss:   308.7657 Validation Accuracy: 0.808594
Epoch 10, Batch 256 -Loss:   236.9510 Validation Accuracy: 0.808594
Epoch 10, Batch 257 -Loss:   360.3816 Validation Accuracy: 0.800781
Epoch 10, Batch 258 -Loss:   316.7054 Validation Accuracy: 0.804688
Epoch 10, Batch 259 -Loss:   511.6769 Validation Accuracy: 0.804688
Epoch 10, Batch 260 -Loss:   364.6937 Validation Accuracy: 0.804688
Epoch 10, Batch 261 -Loss:    90.4264 Validation Accuracy: 0.804688
Epoch 10, Batch 262 -Loss:   270.4311 Validation Accuracy: 0.808594
Epoch 10, Batch 263 -Loss:   236.8204 Validation Accuracy: 0.812500
Epoch 10, Batch 264 -Loss:   301.7567 Validation Accuracy: 0.812500
Epoch 10, Batch 265 -Loss:   252.8448 Validation Accuracy: 0.812500
Epoch 10, Batch 266 -Loss:   317.7320 Validation Accuracy: 0.812500
Epoch 10, Batch 267 -Loss:   319.6678 Validation Accuracy: 0.812500
Epoch 10, Batch 268 -Loss:   323.5330 Validation

Epoch 10, Batch 375 -Loss:   281.1390 Validation Accuracy: 0.816406
Epoch 10, Batch 376 -Loss:   207.4584 Validation Accuracy: 0.816406
Epoch 10, Batch 377 -Loss:   224.7826 Validation Accuracy: 0.816406
Epoch 10, Batch 378 -Loss:   313.8479 Validation Accuracy: 0.816406
Epoch 10, Batch 379 -Loss:   235.3392 Validation Accuracy: 0.820312
Epoch 10, Batch 380 -Loss:   296.0457 Validation Accuracy: 0.816406
Epoch 10, Batch 381 -Loss:   240.8499 Validation Accuracy: 0.812500
Epoch 10, Batch 382 -Loss:   215.8387 Validation Accuracy: 0.812500
Epoch 10, Batch 383 -Loss:   332.1722 Validation Accuracy: 0.812500
Epoch 10, Batch 384 -Loss:   182.7698 Validation Accuracy: 0.812500
Epoch 10, Batch 385 -Loss:   289.2596 Validation Accuracy: 0.812500
Epoch 10, Batch 386 -Loss:   315.3920 Validation Accuracy: 0.816406
Epoch 10, Batch 387 -Loss:   284.9305 Validation Accuracy: 0.816406
Epoch 10, Batch 388 -Loss:   197.8183 Validation Accuracy: 0.816406
Epoch 10, Batch 389 -Loss:   192.5607 Validation